In [76]:
import numpy as np
import pandas as pd

In [77]:
text = np.loadtxt('Knapsack.txt')

In [78]:
text

array([[ 26., 550.],
       [ 91.,  84.],
       [ 72.,  83.],
       [ 90.,  43.],
       [ 46.,   4.],
       [ 55.,  44.],
       [  8.,   6.],
       [ 35.,  82.],
       [ 75.,  92.],
       [ 61.,  25.],
       [ 15.,  83.],
       [ 77.,  56.],
       [ 40.,  18.],
       [ 63.,  58.],
       [ 75.,  14.],
       [ 29.,  48.],
       [ 75.,  70.],
       [ 17.,  96.],
       [ 78.,  32.],
       [ 40.,  68.],
       [ 44.,  92.],
       [ 40.,  60.],
       [ 20.,  55.],
       [ 23.,  12.],
       [ 55.,  15.],
       [ 65.,  38.],
       [ 12.,  40.]])

In [79]:
!pip install -q pyomo
from pyomo.environ import *

In [80]:
model=ConcreteModel()

In [81]:
obj_coef_c = text[1:,0]
constr_coef = text[1:,1]

In [82]:
N = 26
W = 550
col_indices = np.arange(n)

In [83]:
model.x = Var(col_indices,domain=Binary)

In [84]:
model.constraints = ConstraintList()

In [85]:
model.constraints.add(sum(constr_coef[j]*model.x[j] for j in col_indices) <= W)

In [86]:
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=maximize)

In [87]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}

1 Var Declarations
    x : Size=26, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Bin

In [88]:
!apt-get install -y -qq coinor-cbc

In [89]:
opt_cbc = SolverFactory('cbc')

In [90]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)


Problem: 
- Name: unknown
  Lower bound: 911.0
  Upper bound: 911.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 26
  Number of binary variables: 26
  Number of integer variables: 26
  Number of nonzeros: 26
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.02
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.04108691215515137
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.04108691215515137


In [95]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Item ',i+1,' = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  911.0

Decision Variables
Item  1  =  1.0
Item  2  =  1.0
Item  3  =  1.0
Item  4  =  1.0
Item  5  =  1.0
Item  6  =  1.0
Item  7  =  0.0
Item  8  =  0.0
Item  9  =  1.0
Item  10  =  0.0
Item  11  =  1.0
Item  12  =  1.0
Item  13  =  0.0
Item  14  =  1.0
Item  15  =  0.0
Item  16  =  1.0
Item  17  =  0.0
Item  18  =  1.0
Item  19  =  0.0
Item  20  =  0.0
Item  21  =  0.0
Item  22  =  0.0
Item  23  =  1.0
Item  24  =  1.0
Item  25  =  1.0
Item  26  =  0.0

Constraints
constraints : Size=1
    Key : Lower : Body  : Upper
      1 :  None : 544.0 : 550.0


The optimal subset to include in the Knapsack are: \
**Item number** $1,2,3,4,5,6,9,11,12,14,16,18,23,24,25$

## **Dynamic Programming:**

In [92]:
import time
start=time.time()
items=[]
for i in range(1,N+1):
  items.append((text[i,1],text[i,0]))
max_weight = W

# Create a table with n+1 rows and W+1 columns
table = [[0 for w in range(max_weight+1)] for i in range(len(items)+1)]

# Fill the table using dynamic programming
for i in range(1, len(items)+1):
    for w in range(1, max_weight+1):
        if items[i-1][0] > w:
            # Item i cannot be included in the knapsack
            table[i][w] = table[i-1][w]
        else:
            # Find the maximum value with or without item i
            value_with = items[i-1][1] + table[i-1][int(w-items[i-1][0])]
            value_without = table[i-1][w]
            table[i][w] = max(value_with, value_without)

# Find the optimal subset of items to include in the knapsack
included_items = []
i, w = len(items), max_weight
while i > 0 and w > 0:
    if table[i][int(w)] != table[i-1][int(w)]:
        included_items.append(i-1)
        w -= items[i-1][0]
    i -= 1
included_items.reverse()
end = time.time()
print('time taken',end - start)
# Print the results
print("Optimal subset of items to include in the knapsack:")
for item in included_items:
    print("- Item", item+1, ": weight =", items[item][0], "value =", items[item][1])
print("Total value of the knapsack:", table[-1][-1])

time taken 0.026823997497558594
Optimal subset of items to include in the knapsack:
- Item 1 : weight = 84.0 value = 91.0
- Item 2 : weight = 83.0 value = 72.0
- Item 3 : weight = 43.0 value = 90.0
- Item 4 : weight = 4.0 value = 46.0
- Item 5 : weight = 44.0 value = 55.0
- Item 6 : weight = 6.0 value = 8.0
- Item 9 : weight = 25.0 value = 61.0
- Item 11 : weight = 56.0 value = 77.0
- Item 12 : weight = 18.0 value = 40.0
- Item 14 : weight = 14.0 value = 75.0
- Item 16 : weight = 70.0 value = 75.0
- Item 18 : weight = 32.0 value = 78.0
- Item 23 : weight = 12.0 value = 23.0
- Item 24 : weight = 15.0 value = 55.0
- Item 25 : weight = 38.0 value = 65.0
Total value of the knapsack: 911.0


The optimal subset to include in the Knapsack are: \
**Item number** $1,2,3,4,5,6,9,11,12,14,16,18,23,24,25$

We can observe that the time taken to solve the problem: \
By Integer Programming Problem = $0.04108691215515137$ \
By Dynamic Programming = $0.026823997497558594$

Hence **Dynamic Programming** is a better approach than formulating an **Integer Programming** Problem.